</br>
<h1 align="center" style="color:green">Feature Extraction on Meteorological & Dengue Dataset in the City of Manila</br> DOST-Pagasa Port Area Dataset & DOH City of Manila Dengue Dataset</h1>
<div style="text-align:center">Prepared by <b>Jose Rafael C Crisostomo, Jan Vincent G. Elleazar, Dodge Deiniol D. Lapis, and Carl Jacob F. Mateo</b><br>
FOMaC-Autoformer: A Hybrid First Order Markov Chain-Autoformer Model for Dengue Incidence Forecasting in the City of Manila<br>
<b>University of Santo Tomas - College of Information and Computing Sciences</b>
</div>

We will extract all features from both datasets into the following: Temporal Features, Statistical Features, and Composite Features. These will then converted into feature vectors in which allows the Autoformer model to be fed by the right input data.
***

# Temporal Features

In [ ]:
# Insert code for extracting temporal features

***
# Statistical Features

In [ ]:
# Insert code for extracting statistical features

***
# Composite Features

In [ ]:
# Insert code for extracting composite features